# PREDICTION: Deep Learning Forecasting with Feature Selection

In this section, we implement and evaluate deep learning models to predict diagnostic activity over time, based on the selected set of input features derived from previous correlation and VIF analyses. The objective is to assess the effectiveness of using carefully filtered, non-redundant variables in time-series forecasting tasks within the primary care setting.

Building on the previous steps — where redundant diagnoses were removed to ensure interpretability and reduce multicollinearity — we now train neural network architectures that can capture temporal dependencies and complex non-linear relationships in the data.

This phase involves:

- 1. Model Definition: Selection and configuration of a deep learning architecture (e.g., LSTM, GRU, or Transformer) suitable for univariate or multivariate time-series prediction.

- 2. Training & Evaluation: Execution of training pipelines with the filtered feature set, and performance evaluation using appropriate metrics (e.g., MAE, RMSE, R²) on both training and validation sets.

- 3. Forecasting Across Horizons: The models are tested on different prediction horizons to understand their robustness and sensitivity to time-window configuration.

- 4. Comparison with Baselines: Results are compared against baseline models (e.g., naive persistence or linear regression) to quantify the added value of using deep learning with feature selection.

This strategy aims to demonstrate that simpler yet informative inputs, when paired with powerful predictive models, can offer accurate and actionable forecasts — potentially supporting early-warning systems and demand planning in healthcare.

In [ ]:
# no warnings
import warnings
warnings.filterwarnings('ignore')
# no logging
import logging
logging.disable(logging.CRITICAL)


import pandas as pd
import numpy as np
import math
import seaborn as sns
import sys
import time
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import pydot
import graphviz

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.tsa.seasonal import seasonal_decompose

import scipy
import scipy.stats
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import torch.optim as optim

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
# Set random seed for reproducibility
tf.random.set_seed(1234)
from keras import backend as K
from keras.models import Model, Sequential

## 0. DATA RENDERING FUNCTIONS

In [ ]:
# smoother function ---------------------------------
#       (revisar --> rolling(funció per fer-ho més fàcil))
def smoother(df,window_size):
    smoothed = {}
    for column in df.columns:
        if column != 'visi_data_visita':
            column_list = []
            for i in range(len(df)):
                if i == 0:
                    column_list.append(df[column].iloc[0])
                elif i < window_size:
                    column_list.append(df[column].iloc[:i+1].mean())
                else:
                    column_list.append(df[column].iloc[i-window_size:i+1].mean())
            smoothed[column] = column_list

    smoothed = pd.DataFrame.from_dict(smoothed)
    smoothed.set_index(df.index, inplace=True)
    return smoothed

# plot example 10 diags -------------------------
def plot_example(df, title):
    """
    Randomly selects and plots 10 time series from a DataFrame.
    
    Parameters:
    - df (pd.DataFrame): Time-indexed DataFrame with multiple columns (e.g. diagnostics).
    - title (str): Title for the plot.
    """
    # Randomly sample 10 columns
    sampled_cols = np.random.choice(df.columns, size=10, replace=False)
    dff = df[sampled_cols].copy()

    # Add date column
    dff["date"] = dff.index

    # Melt and plot
    sns.set(rc={'figure.figsize': (20, 8)})
    sns.lineplot(
        data=dff.melt(id_vars=['date']),
        x='date', y='value', hue='variable'
    ).set(title=title)

    plt.xlabel("Date")
    plt.ylabel("Value")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# plot example 10 diags -------------------------
def ploter(df,title,n,code):
    # PLOT RAW DATA (example 10 diags)
    cols = list(df.columns)[0:n]
    cols.append(code)
    dff = df[cols]
    dff["date"] = dff.index
    sns.set(rc={'figure.figsize':(20,8)})
    sns.lineplot(data=dff.replace('nan', float('nan')).melt(id_vars=['date']),x='date', y='value', hue='variable').set(title=title)

In [ ]:
# RAW DATA --------------------------------------------------------------
df = pd.read_csv('synthetic_timeseries.csv', index_col=0)
df.index = pd.date_range(start="2010-01-01", periods=len(df), freq="D")
df = df.clip(lower=0)

window_size = 14
smoothed = smoother(df, window_size)

smoothed_scaled = (smoothed-smoothed.min())/(smoothed.max()-smoothed.min()) # normalize df

smoothed = smoothed_scaled

In [ ]:
# SMOOTHED DATA --------------------------------------------------------
window_size = 30
# ----------------------------------------------------------------------
plot_example(smoothed,"SMOOTHED (example 10 ts)")

# DEEP LEARNING LAYER

## 0. NEEDED FUNCTIONS

In [ ]:
# Split the time series into chunk of the correct size. 
def split_sequence(sequence, look_back, forecast_horizon):
    X, y = list(), list()
    for i in range(len(sequence)): 
        lag_end = i + look_back
        forecast_end = lag_end + forecast_horizon
        if forecast_end > len(sequence):
            break
        seq_x, seq_y = sequence[i:lag_end], sequence[lag_end:forecast_end]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


# The used callbacks while compiling the models are the following. ModelCheckpoint is to save the model(weights) 
# at certain frequencies. EarlyStopping is used for stopping the progress if the monitored evaluation metric is 
# no longer improved. ReduceLROnPlateau is for decreasing the learning rate when the monitored metric has stopped 
# improving.
checkpoint_filepath = '../callbacks/best_model.keras'

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.005,
    patience=10,
    mode='min'
)

rlrop_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, mode='min', patience=3, min_lr=0.001)


# transform data back to the original scale to evaluate the forecast. 
# We might simply utilize the following method. 
def inverse_transform(y_test, yhat):
    
    y_test_reshaped = y_test.reshape(-1, y_test.shape[-1])
    y_test_inverse = scaler.inverse_transform(y_test_reshaped)
    y_test_inverse = y_test_inverse.reshape(y_test.shape[0], -1)[:, :-1] #last col
    
    yhat_reshaped = yhat.reshape(-1, yhat.shape[-1]) 
    yhat_inverse = scaler.inverse_transform(yhat_reshaped)
    yhat_inverse = yhat_inverse.reshape(yhat.shape[0], -1)[:, :-1]
    
    return y_test_inverse, yhat_inverse

# To evaluate the forecast: mean square error(mse), mean absolute error(mae), 
# and mean absolute percentage error(mape) 
def evaluate_forecast(y_test_inverse, yhat_inverse):
    mse_ = tf.keras.losses.MeanSquaredError()
    mae_ = tf.keras.losses.MeanAbsoluteError()
    mape_ = tf.keras.losses.MeanAbsolutePercentageError() 
    mae = mae_(y_test_inverse,yhat_inverse)
    print('mae:', mae)
    mse = mse_(y_test_inverse,yhat_inverse)
    print('mse:', mse)
    mape = mape_(y_test_inverse,yhat_inverse)
    print('mape:', mape)
    
def get_results(y_test_inverse, yhat_inverse):
    mse_ = tf.keras.losses.MeanSquaredError()
    mae_ = tf.keras.losses.MeanAbsoluteError()
    mape_ = tf.keras.losses.MeanAbsolutePercentageError() 
    mae = mae_(y_test_inverse,yhat_inverse)
    mse = mse_(y_test_inverse,yhat_inverse)
    mape = mape_(y_test_inverse,yhat_inverse)

    return mae, mse, mape

# plot model results ........................................................
def plt_model(y_test_inverse,yhat_inverse, model):
    plt.figure(figsize=(20, 10))
    plt.plot(pd.DataFrame(y_test_inverse)[[col_idx]], label='True Values')
    plt.plot(pd.DataFrame(yhat_inverse)[[col_idx]], label='Predicted Values')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.title('Real vs. Predicted Values // MODEL: '+model)
    plt.legend()
    plt.show()

## 1. MODELS

Layers: 
- *TimeDistributed layer*: is a kind of wrapper and expects another layer as an argument. It applies this layer to every temporal slice of input and therefore allows to build models that have one-to-many, many-to-many architectures. Similarly, it expects the input at least as 3 dimensions. (https://stackoverflow.com/questions/51749404/how-to-connect-lstm-layers-in-keras-repeatvector-or-return-sequence-true)

- *RepeatVector*: basically repeats inputs n times. In other words, it increases the dimension of the output shape by 1. (https://github.com/keras-team/keras/issues/1029)

#### 1.1. GRU architecture

#### 1.1. LSTM architecture

#### 1.1. Bi-directional LSTM architecture

#### 1.1. Encoder-Decoder LSTM architecture
- RNN Encoder-Decoder, consists of two recurrent neural networks (RNN) that act as an encoder and a decoder pair. The encoder maps a variable-length source sequence to a fixed-length vector, and the decoder maps the vector representation back to a variable-length target sequence. Seq2Seq sequences (vary length in input and output). Many to many architecture.
https://machinelearningmastery.com/encoder-decoder-long-short-term-memory-networks/

#### 1.2. CNN-LSTM Encoder-Decoder Model (https://ieeexplore.ieee.org/document/9587207)
- The following model is an extension of encoder-decoder architecture where the encoder part consists of Conv1D layers, unlike the previous model. First of all, two subsequent Conv1D layers are placed at the beginning to extract features, and then it is flattened after pooling the results of Conv1D. The rest of the architecture is very similar to the previous model.

#### 1.3. Vector Output Model
- This architecture might be thought of as a much more common architecture compared to the above-mentioned models, however, it is not very suitable for our case. Nevertheless, I share an example model to give an idea. At variance with encoder-decoder architecture, neither RepeatVector nor TimeDistributed layer exists. The point is to add a Dense layer with FORECAST_RANGE*n_features node, and then reshape it accordingly at the next layer. You might also design a similar architecture with RepeatVector layer instead of Reshape layer. The time steps of each series would be flattened in this structure and must interpret each of the outputs as a specific time step for a specific series during training and prediction. That means we also might reshape our label set as 2 dimensions rather than 3 dimensions, and interpret the results in the output layer accordingly without using Reshape layer. For simplicity, I did not change the shape of the label set, but just keep it in mind this alternative way as well. I also utilized Conv1D layers at the beginning of the architecture.
- This structure might be also called a multi-channel model. Do not overestimate the names. When dealing with multiple time series, CNNs with multiple channels are utilized traditionally. In this structure, each channel corresponds to a single time series and similarly extracts convolved features separately for each time series. Since all of the extracted features are combined before feeding into the LSTM layer, some typical features of each time series might be lost.

#### 1.4. Multi-Head CNN-LSTM Model
- This architecture is a bit different from the above-mentioned models. It is explained very clearly in the study of Canizo. The multi-head structure uses multiple one-dimensional CNN layers in order to process each time series and extract independent convolved features from each time series. These separate CNNs are called “head” and flattened, concatenated, and reshaped respectively before feeding into the LSTM layer. To summarize, multi-head structures utilize multiple CNNs rather than only one CNN like in multi-channel structure. Therefore, they might be more successful to keep significant features of each time series and make better forecasts in this sense. (https://www.sciencedirect.com/science/article/abs/pii/S0925231219309877)

In [ ]:
# fit/train model ..........................................................
def fit_model(model, X_train, y_train, epochs, batch_size, validation, patience, verbose=1):
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = patience)
    history = model.fit(X_train, y_train, epochs = epochs,  validation_split = validation, 
                        batch_size = batch_size, shuffle = False, verbose= verbose,
                        callbacks=[early_stopping_callback, checkpoint_callback, rlrop_callback])
    return history

# predict ..................................................................
def prediction(model,X_test):
    prediction = model.predict(X_test)
    return prediction

# .......................................
# ---- MODELS ---------------------------
# .......................................

# model 1: GRU unit --------------------------------------------------------------
def create_model_gru(X_train, optimizer='adam'):
    
    model = Sequential()
    model.add(GRU(units = 100, input_shape = [X_train.shape[1], X_train.shape[2]]))
    model.add(RepeatVector(FORECAST_RANGE))
    model.add(Dropout(0.2))
    model.add(GRU(units = 100, return_sequences = True,))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(n_features)))
    
    model.compile(loss='mse', optimizer=optimizer)
    
    return model

# model 2: LSTM unit --------------------------------------------------------------
def create_model_lstm(X_train, optimizer='adam'):
    
    model = Sequential()
    model.add(LSTM(units = 100, input_shape = [X_train.shape[1], X_train.shape[2]]))
    model.add(RepeatVector(FORECAST_RANGE))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 100, return_sequences = True,))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(n_features)))
    
    model.compile(loss='mse', optimizer=optimizer)
    
    return model

# model 3: BILSTM unit --------------------------------------------------------------
def create_model_bilstm(X_train, optimizer='adam'):
    
    model = Sequential()
    model.add(Bidirectional(LSTM(units = 100), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(RepeatVector(FORECAST_RANGE))
    model.add(Bidirectional(LSTM(units = 100, return_sequences=True)))
    model.add(TimeDistributed(Dense(n_features)))
    
    model.compile(loss='mse', optimizer=optimizer)
    
    return model

# model 4: ENCODER-DECODER LSTM unit ------------------------------------------------
def create_model_enc_dec(X_train, optimizer='adam'):
    
    model_enc_dec = Sequential()
    model_enc_dec.add(LSTM(100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model_enc_dec.add(RepeatVector(FORECAST_RANGE))
    model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
    model_enc_dec.add(TimeDistributed(Dense(n_features)))
    
    model_enc_dec.compile(optimizer=optimizer, loss='mse')
    
    return model_enc_dec

# model 4: ENCODER-DECODER CNN unit ------------------------------------------------

#The kernel_size determines the spatial extent of the local region that is being considered for feature extraction. 
# A larger kernel size captures a broader context but may result in a higher number of parameters. 
# A smaller kernel size captures finer details but may require more layers to learn complex patterns. 
# Common values for kernel_size are 3x3, 5x5, or 7x7.
#   -  Smaller Kernel (e.g., 3x3): Captures fine-grained features, well-suited for capturing small patterns and edges.
#   -  Larger Kernel (e.g., 5x5 or 7x7): Captures more global features and spatial hierarchies.
#   -  Smaller kernels may require deeper networks to capture complex patterns.
#   -  Larger kernels may result in more parameters and computational cost.

def create_model_enc_dec_cnn(X_train, optimizer='adam',kern_size = 3):
    
    model_enc_dec_cnn = Sequential()
    model_enc_dec_cnn.add(Conv1D(filters=64, kernel_size=kern_size, activation='relu', 
                                 input_shape=(X_train.shape[1], X_train.shape[2])))
    model_enc_dec_cnn.add(Conv1D(filters=64, kernel_size=kern_size, activation='relu'))
    model_enc_dec_cnn.add(MaxPooling1D(pool_size=2))
    model_enc_dec_cnn.add(Flatten())
    model_enc_dec_cnn.add(RepeatVector(FORECAST_RANGE))
    model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
    model_enc_dec_cnn.add(TimeDistributed(Dense(100, activation='relu')))
    model_enc_dec_cnn.add(TimeDistributed(Dense(n_features)))
    
    model_enc_dec_cnn.compile(loss='mse', optimizer=optimizer)
    
    return model_enc_dec_cnn


# model 5: VECTOR OUTPUT -----------------------------------------------------------
def create_model_vector_output(X_train, optimizer='adam',kern_size = 3):
    
    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2])) 
    conv = Conv1D(filters=4, kernel_size=kern_size, activation='relu')(input_layer)
    conv = Conv1D(filters=6, kernel_size=kern_size, activation='relu')(conv)

    lstm = LSTM(100, return_sequences=True, activation='relu')(conv)
    dropout = Dropout(0.2)(lstm)
    lstm = LSTM(100, activation='relu')(dropout)
    dense = Dense(FORECAST_RANGE*n_features, activation='relu')(lstm)
    output_layer = Reshape((FORECAST_RANGE,n_features))(dense)
    model_vector_output = Model([input_layer], [output_layer])
    
    model_vector_output.compile(optimizer=optimizer, loss='mse')
    
    return model_vector_output

# model 6: Multi head LSTM cnn -----------------------------------------------------------
def create_model_multi_head_cnn_lstm(X_train, optimizer='adam',kern_size = 3):
    
    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))  #Look back, n_features
    head_list = []
    for i in range(0, n_features):
        conv_layer_head = Conv1D(filters=4, kernel_size=kern_size, activation='relu')(input_layer)
        conv_layer_head_2 = Conv1D(filters=6, kernel_size=kern_size, activation='relu')(conv_layer_head)
        conv_layer_flatten = Flatten()(conv_layer_head_2)
        head_list.append(conv_layer_flatten)

    concat_cnn = Concatenate(axis=1)(head_list)
    reshape = Reshape((head_list[0].shape[1], n_features))(concat_cnn)
    lstm = LSTM(100, activation='relu')(reshape)
    repeat = RepeatVector(FORECAST_RANGE)(lstm)
    lstm_2 = LSTM(100, activation='relu', return_sequences=True)(repeat)
    dropout = Dropout(0.2)(lstm_2)
    dense = Dense(n_features, activation='linear')(dropout)
    multi_head_cnn_lstm_model = Model(inputs=input_layer, outputs=dense)
    
    multi_head_cnn_lstm_model.compile(optimizer=optimizer, loss='mse')
    
    return multi_head_cnn_lstm_model

### Define imputs

In [ ]:
# Objective and predictors ...................................
objective = "timeseries_350"
cc_predictors = ['timeseries_405','timeseries_363','timeseries_975','timeseries_120','timeseries_2','timeseries_541','timeseries_775',
                 'timeseries_181','timeseries_692','timeseries_443','timeseries_763','timeseries_31','timeseries_324'] 

# df definition ..............................................
cc_predictors.append(objective)
variables_predictors = cc_predictors.copy()
subdf = smoothed[cc_predictors]

# Add temporality (weekday and month) ........................
temporality = False
if temporality:
    subdf['weekday'] = subdf['date'].dt.day_name()
    subdf['month'] = subdf['date'].dt.month_name()
    dummies_day = pd.get_dummies(subdf['weekday'], prefix='weekday_')
    dummies_month = pd.get_dummies(subdf['month'], prefix='month_')
    dummies_day = dummies_day.astype(int)
    dummies_month = dummies_month.astype(int)
    subdf = pd.concat([subdf, dummies_day, dummies_month], axis=1)
    subdf = subdf.drop('weekday', axis = 1)
    subdf = subdf.drop('month', axis = 1)
    cc_predictors = cc_predictors + dummies_day.columns.tolist()
    cc_predictors = cc_predictors + dummies_month.columns.tolist()
    cc_predictors.remove('date')

# put objective to last column ..............................
cc_predictors = [col for col in subdf.columns if col != objective]
cc_predictors.append(objective)
subdf = subdf[cc_predictors]
display(subdf) 

# train-test split ...........................................
train_size = int(len(subdf)*0.8)
train_dataset, test_dataset = subdf.iloc[5:train_size],subdf.iloc[train_size:]

# Plot train and test data ...................................
plt.figure(figsize = (10, 4))
plt.plot(train_dataset[objective])
plt.plot(test_dataset[objective])
plt.xlabel('Time (day)')
plt.ylabel('n diags Hipertension')
plt.legend(['Train set', 'Test set'], loc='upper right')

print('Dimension of train data: ',train_dataset.shape)
print('Dimension of test data: ', test_dataset.shape)

#scale ......................................................
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train_dataset)
scaled_test = scaler.transform(test_dataset)

# get idx of objective variable
col_idx = subdf.columns.get_loc(objective)

### DL parameters

In [ ]:
LOOK_BACK =  30 #731 (2 anys)     # quants dies amb anterioritat mires abans de predir
FORECAST_RANGE = 8                # quants dies em de predir
n_features = len(cc_predictors)   # quantes variables (sistema molt sensible)
  
epochs = 10#0
batch_size = 128
validation = 0.1
patience = 10

X_train, y_train = split_sequence(scaled_train, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
X_test, y_test = split_sequence(scaled_test, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

### MODEL EVALUATION

In [ ]:
# GRU model .........................................................................................................
print('\n >>>>> model 1: GRU')
model_gru = create_model_gru(X_train)
history = fit_model(model_gru,X_train, y_train, epochs, batch_size, validation, patience)
yhat_gru = prediction(model_gru,X_test)
y_test_inverse, yhat_gru_inverse = inverse_transform(y_test, yhat_gru)
evaluate_forecast(y_test_inverse, yhat_gru_inverse)
plt_model(y_test_inverse, yhat_gru_inverse,"GRU")

# LSTM model ........................................................................................................
print('\n >>>>> model 2: LSTM')
model_lstm = create_model_lstm(X_train)
history = fit_model(model_lstm,X_train, y_train, epochs, batch_size, validation, patience)
yhat_lstm = prediction(model_lstm,X_test)
y_test_inverse, yhat_lstm_inverse = inverse_transform(y_test, yhat_lstm)
evaluate_forecast(y_test_inverse, yhat_lstm_inverse)
plt_model(y_test_inverse, yhat_lstm_inverse,"LSTM")

# Bi-directional model .............................................................................................
print('\n >>>>> model 3: Bi-directional')
model_bilstm = create_model_bilstm(X_train)
history = fit_model(model_bilstm,X_train, y_train, epochs, batch_size, validation, patience)
yhat_bilstm = prediction(model_bilstm,X_test)
y_test_inverse, yhat_bilstm_inverse = inverse_transform(y_test, yhat_bilstm)
evaluate_forecast(y_test_inverse, yhat_bilstm_inverse)
plt_model(y_test_inverse, yhat_bilstm_inverse,"Bi-directional_LSTM")

# Encoder-decoder LSTM model .......................................................................................
print('\n >>>>> model 4: Encoder-decoder LSTM')
model_enc_dec = create_model_enc_dec(X_train)
history = fit_model(model_enc_dec,X_train, y_train, epochs, batch_size, validation, patience)
yhat_endelstm = prediction(model_enc_dec,X_test)
y_test_inverse, yhat_endelstm_inverse = inverse_transform(y_test, yhat_endelstm)
evaluate_forecast(y_test_inverse, yhat_endelstm_inverse)
plt_model(y_test_inverse, yhat_endelstm_inverse,"ENCODER_DECODER_LSTM")

# CNN-LSTM Encoder-Decoder model ...................................................................................
print('\n >>>>> model 5: CNN-LSTM Encoder-Decoder')
model_enc_dec_cnn = create_model_enc_dec_cnn(X_train)
history = fit_model(model_enc_dec_cnn, X_train, y_train, epochs, batch_size, validation, patience)
yhat_cnnlstmende = prediction(model_enc_dec_cnn,X_test)
y_test_inverse, yhat_cnnlstmende_inverse = inverse_transform(y_test, yhat_cnnlstmende)
evaluate_forecast(y_test_inverse, yhat_cnnlstmende_inverse)
plt_model(y_test_inverse, yhat_cnnlstmende_inverse,"Encoder_DECODER_CNN_LSTM")

# Vector-Output model .............................................................................................
print('\n >>>>> model 6: Vector-Output')
model_vector_output = create_model_vector_output(X_train)
history = fit_model(model_vector_output, X_train, y_train, epochs, batch_size, validation, patience)
yhat_veout = prediction(model_vector_output,X_test)
y_test_inverse, yhat_veout_inverse = inverse_transform(y_test, yhat_veout)
evaluate_forecast(y_test_inverse, yhat_veout_inverse)
plt_model(y_test_inverse, yhat_veout_inverse,"Encoder_DECODER_CNN_LSTM")

# Multi-Head CNN-LSTM model ......................................................................................
print('\n >>>>> model 7: Multi-Head CNN-LSTM')
multi_head_cnn_lstm_model = create_model_multi_head_cnn_lstm(X_train)
history = fit_model(multi_head_cnn_lstm_model, X_train, y_train, epochs, batch_size, validation, patience)
yhat_muhecnnlstm = prediction(multi_head_cnn_lstm_model,X_test)
y_test_inverse, yhat_muhecnnlstm_inverse = inverse_transform(y_test, yhat_muhecnnlstm)
evaluate_forecast(y_test_inverse, yhat_muhecnnlstm_inverse)
plt_model(y_test_inverse, yhat_muhecnnlstm_inverse, "Multi-Head CNN-LSTM")  

### Get results all together

In [ ]:
# GRU model .........................................................................................................
print('\n >>>>> model 1: GRU')
evaluate_forecast(y_test_inverse, yhat_gru_inverse)
#plt_model(y_test_inverse, yhat_gru_inverse,"GRU")

# LSTM model ........................................................................................................
print('\n >>>>> model 2: LSTM')
evaluate_forecast(y_test_inverse, yhat_lstm_inverse)
#plt_model(y_test_inverse, yhat_lstm_inverse,"LSTM")

# Bi-directional model .............................................................................................
print('\n >>>>> model 3: Bi-directional')
evaluate_forecast(y_test_inverse, yhat_bilstm_inverse)
#plt_model(y_test_inverse, yhat_bilstm_inverse,"Bi-directional_LSTM")

# Encoder-decoder LSTM model .......................................................................................
print('\n >>>>> model 4: Encoder-decoder LSTM')
evaluate_forecast(y_test_inverse, yhat_endelstm_inverse)
#plt_model(y_test_inverse, yhat_endelstm_inverse,"ENCODER_DECODER_LSTM")

# CNN-LSTM Encoder-Decoder model ...................................................................................
print('\n >>>>> model 5: CNN-LSTM Encoder-Decoder')
evaluate_forecast(y_test_inverse, yhat_cnnlstmende_inverse)
#plt_model(y_test_inverse, yhat_cnnlstmende_inverse,"Encoder_DECODER_CNN_LSTM")

# Vector-Output model .............................................................................................
print('\n >>>>> model 6: Vector-Output')
evaluate_forecast(y_test_inverse, yhat_veout_inverse)
#plt_model(y_test_inverse, yhat_veout_inverse,"Encoder_DECODER_CNN_LSTM")

# Multi-Head CNN-LSTM model ......................................................................................
print('\n >>>>> model 7: Multi-Head CNN-LSTM')
evaluate_forecast(y_test_inverse, yhat_muhecnnlstm_inverse)
#plt_model(y_test_inverse, yhat_muhecnnlstm_inverse, "Multi-Head CNN-LSTM")  

# PLOT ----------------------------------------------------------------------------------------------------------
# plot models
plt.figure(figsize=(20, 10))
plt.plot(pd.DataFrame(y_test_inverse)[[col_idx]], label='True Values', linewidth=2.5)
plt.plot(pd.DataFrame(yhat_gru_inverse)[[col_idx]], label='Pred. GRU', linestyle='--')
plt.plot(pd.DataFrame(yhat_lstm_inverse)[[col_idx]], label='Pred. LSTM', linestyle='--')
plt.plot(pd.DataFrame(yhat_bilstm_inverse)[[col_idx]], label='Pred. BiLSTM', linestyle='--')
plt.plot(pd.DataFrame(yhat_endelstm_inverse)[[col_idx]], label='Pred. Enc-Dec LSTM', linestyle='--')
plt.plot(pd.DataFrame(yhat_cnnlstmende_inverse)[[col_idx]], label='Pred. CNN-LSTM Enc-Dec', linestyle='--')
plt.plot(pd.DataFrame(yhat_veout_inverse)[[col_idx]], label='Pred. Vector Output', linestyle='--')
plt.plot(pd.DataFrame(yhat_muhecnnlstm_inverse)[[col_idx]], label='Pred. Mult-Head CNN-LSTM', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Real vs. Predicted Values MODELS')
plt.legend()
plt.show()

### grid search params

In [ ]:
def auto_grid_search(model):
    import tensorflow as tf

    # Define search space inside
    epochs_list = [10, 20, 30, 50, 100, 200, 500, 1000] 
    bt_size = [10, 20, 40, 60, 80, 100, 200, 500, 1000]
    opti = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

    var1, var2, var3 = 'epochs', 'batch_size', 'optimizer'
    mapes = []

    # Grid search epochs
    for epo in epochs_list:
        model_ = model(X_train)
        history = fit_model(model_, X_train, y_train, epo, 128, validation, patience, verbose=0)
        yhat_ = prediction(model_, X_test)
        y_test_inverse, yhat_inverse = inverse_transform(y_test, yhat_)
        mape = tf.keras.losses.MeanAbsolutePercentageError()(y_test_inverse, yhat_inverse)
        mapes.append(mape)
        print(f'mape: {mape:.4f}  // model: {epo} {var1}')
    best_epochs = epochs_list[mapes.index(min(mapes))]
    print(f"BEST: {min(mapes):.4f} USING {best_epochs} {var1}")

    # Grid search batch size
    mapes = []
    for bt in bt_size:
        model_ = model(X_train)
        history = fit_model(model_, X_train, y_train, best_epochs, bt, validation, patience, verbose=0)
        yhat_ = prediction(model_, X_test)
        y_test_inverse, yhat_inverse = inverse_transform(y_test, yhat_)
        mape = tf.keras.losses.MeanAbsolutePercentageError()(y_test_inverse, yhat_inverse)
        mapes.append(mape)
        print(f'mape: {mape:.4f}  // model: {bt} {var2}')
    best_bt = bt_size[mapes.index(min(mapes))]
    print(f"BEST: {min(mapes):.4f} USING {best_bt} {var2} (and {best_epochs} epochs)")

    # Grid search optimizer
    mapes = []
    for opt in opti:
        model_ = model(X_train, optimizer=opt)
        history = fit_model(model_, X_train, y_train, best_epochs, best_bt, validation, patience, verbose=0)
        yhat_ = prediction(model_, X_test)
        y_test_inverse, yhat_inverse = inverse_transform(y_test, yhat_)
        mape = tf.keras.losses.MeanAbsolutePercentageError()(y_test_inverse, yhat_inverse)
        mapes.append(mape)
        print(f'mape: {mape:.4f}  // model: {opt} {var3}')
    best_opt = opti[mapes.index(min(mapes))]
    print(f"BEST: {min(mapes):.4f} USING {best_opt} {var3} (and {best_epochs} epochs and {best_bt} batch_size)")

In [ ]:
print('\n>>> GRID SEARCH PARAMETERS ... GRU model')
auto_grid_search(create_model_gru)
print('\n>>> GRID SEARCH PARAMETERS ... LSTM model')
auto_grid_search(create_model_lstm)
print('\n>>> GRID SEARCH PARAMETERS ... Bi-LSTM model')
auto_grid_search(create_model_bilstm)
print('\n>>> GRID SEARCH PARAMETERS ...encoder-decoder model')
auto_grid_search(create_model_enc_dec)
print('\n>>> GRID SEARCH PARAMETERS ... encoder-decoder cnn model')
auto_grid_search(create_model_enc_dec_cnn)
print('\n>>> GRID SEARCH PARAMETERS ... vector output model')
auto_grid_search(create_model_vector_output)
print('\n>>> GRID SEARCH PARAMETERS ... multi head cnn lstm model')
auto_grid_search(create_model_multi_head_cnn_lstm)

# RUN ALL WITH CORRECT PARAMS AND DIFFERENT LOOK BACK TIME

In [ ]:
FORECAST_RANGE = 7                 # quants dies em de predir
n_features = len(cc_predictors)   # quantes variables (sistema molt sensible)

validation = 0.1
patience = 10
LOOK_BACKS = [7,14,30,60,90,180,365,730]

for LOOK_BACK in LOOK_BACKS:
    
    print("\n \n \n >>>> LOOK BACK PARAMETER: %s" % (LOOK_BACK))
    
    X_train, y_train = split_sequence(scaled_train, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
    X_test, y_test = split_sequence(scaled_test, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
    
    # GRU model .........................................................................................................
    print('\n >>>>> model 1: GRU')
    model_gru = create_model_gru(X_train, optimizer='Nadam')
    history = fit_model(model_gru,X_train, y_train,100, 80, validation, patience)
    yhat_gru = prediction(model_gru,X_test)
    y_test_inverse, yhat_gru_inverse = inverse_transform(y_test, yhat_gru)

    # LSTM model ........................................................................................................
    print('\n >>>>> model 2: LSTM')
    model_lstm = create_model_lstm(X_train, optimizer='Nadam')
    history = fit_model(model_lstm,X_train, y_train, 30, 80, validation, patience)
    yhat_lstm = prediction(model_lstm,X_test)
    y_test_inverse, yhat_lstm_inverse = inverse_transform(y_test, yhat_lstm)

    # Bi-directional model .............................................................................................
    print('\n >>>>> model 3: Bi-directional')
    model_bilstm = create_model_bilstm(X_train, optimizer='adam')
    history = fit_model(model_bilstm,X_train, y_train, 100, 500, validation, patience)
    yhat_bilstm = prediction(model_bilstm,X_test)
    y_test_inverse, yhat_bilstm_inverse = inverse_transform(y_test, yhat_bilstm)

    # Encoder-decoder LSTM model .......................................................................................
    print('\n >>>>> model 4: Encoder-decoder LSTM')
    model_enc_dec = create_model_enc_dec(X_train, optimizer='Adamax')
    history = fit_model(model_enc_dec,X_train, y_train, 200, 10, validation, patience)
    yhat_endelstm = prediction(model_enc_dec,X_test)
    y_test_inverse, yhat_endelstm_inverse = inverse_transform(y_test, yhat_endelstm)

    # CNN-LSTM Encoder-Decoder model ...................................................................................
    print('\n >>>>> model 5: CNN-LSTM Encoder-Decoder')
    model_enc_dec_cnn = create_model_enc_dec_cnn(X_train, optimizer='Nadam',kern_size=3)
    history = fit_model(model_enc_dec_cnn, X_train, y_train, 20, 500, validation, patience)
    yhat_cnnlstmende = prediction(model_enc_dec_cnn,X_test)
    y_test_inverse, yhat_cnnlstmende_inverse = inverse_transform(y_test, yhat_cnnlstmende)

    # Vector-Output model .............................................................................................
    print('\n >>>>> model 6: Vector-Output')
    model_vector_output = create_model_vector_output(X_train, optimizer='Adam',kern_size=3)
    history = fit_model(model_vector_output, X_train, y_train, 50, 10, validation, patience)
    yhat_veout = prediction(model_vector_output,X_test)
    y_test_inverse, yhat_veout_inverse = inverse_transform(y_test, yhat_veout)

    # Multi-Head CNN-LSTM model ......................................................................................
    print('\n >>>>> model 7: Multi-Head CNN-LSTM')
    multi_head_cnn_lstm_model = create_model_multi_head_cnn_lstm(X_train, optimizer='Nadam',kern_size=3)
    history = fit_model(multi_head_cnn_lstm_model, X_train, y_train, 500, 40, validation, patience)
    yhat_muhecnnlstm = prediction(multi_head_cnn_lstm_model,X_test)
    y_test_inverse, yhat_muhecnnlstm_inverse = inverse_transform(y_test, yhat_muhecnnlstm)

        # GRU model .........................................................................................................
    print('\n >>>>> model 1: GRU')
    evaluate_forecast(y_test_inverse, yhat_gru_inverse)
    print('\n >>>>> model 2: LSTM')
    evaluate_forecast(y_test_inverse, yhat_lstm_inverse)
    print('\n >>>>> model 3: Bi-directional')
    evaluate_forecast(y_test_inverse, yhat_bilstm_inverse)
    print('\n >>>>> model 4: Encoder-decoder LSTM')
    evaluate_forecast(y_test_inverse, yhat_endelstm_inverse)
    print('\n >>>>> model 5: CNN-LSTM Encoder-Decoder')
    evaluate_forecast(y_test_inverse, yhat_cnnlstmende_inverse)
    print('\n >>>>> model 6: Vector-Output')
    evaluate_forecast(y_test_inverse, yhat_veout_inverse)
    print('\n >>>>> model 7: Multi-Head CNN-LSTM')
    evaluate_forecast(y_test_inverse, yhat_muhecnnlstm_inverse)

    # PLOT ----------------------------------------------------------------------------------------------------------
    # plot models
    plt.figure(figsize=(20, 10))
    plt.plot(pd.DataFrame(y_test_inverse)[[col_idx]], label='True Values', linewidth=2.5)
    plt.plot(pd.DataFrame(yhat_gru_inverse)[[col_idx]], label='Pred. GRU', linestyle='--')
    plt.plot(pd.DataFrame(yhat_lstm_inverse)[[col_idx]], label='Pred. LSTM', linestyle='--')
    plt.plot(pd.DataFrame(yhat_bilstm_inverse)[[col_idx]], label='Pred. BiLSTM', linestyle='--')
    plt.plot(pd.DataFrame(yhat_endelstm_inverse)[[col_idx]], label='Pred. Enc-Dec LSTM', linestyle='--')
    plt.plot(pd.DataFrame(yhat_cnnlstmende_inverse)[[col_idx]], label='Pred. CNN-LSTM Enc-Dec', linestyle='--')
    plt.plot(pd.DataFrame(yhat_veout_inverse)[[col_idx]], label='Pred. Vector Output', linestyle='--')
    plt.plot(pd.DataFrame(yhat_muhecnnlstm_inverse)[[col_idx]], label='Pred. Mult-Head CNN-LSTM', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.title('Real vs. Predicted Values MODELS')
    plt.legend()
    plt.show()

In [ ]:
FORECAST_RANGE = [1,7,14,21,28]                 # quants dies em de predir
n_features = len(cc_predictors)   # quantes variables (sistema molt sensible)

validation = 0.1
patience = 10
LOOK_BACKS = [7,14,30,60,90,180,365,730]

for LOOK_BACK in LOOK_BACKS:
    for FORECAST in FORECAST_RANGE: 
    
        print("\n \n \n >>>> LOOK BACK PARAMETER: %s" % (LOOK_BACK))

        X_train, y_train = split_sequence(scaled_train, look_back=LOOK_BACK, forecast_horizon=FORECAST)
        X_test, y_test = split_sequence(scaled_test, look_back=LOOK_BACK, forecast_horizon=FORECAST)

        # GRU model .........................................................................................................
        print('\n >>>>> model 1: GRU')
        model_gru = create_model_gru(X_train, optimizer='Nadam')
        history = fit_model(model_gru,X_train, y_train,100, 80, validation, patience)
        yhat_gru = prediction(model_gru,X_test)
        y_test_inverse, yhat_gru_inverse = inverse_transform(y_test, yhat_gru)

        # LSTM model ........................................................................................................
        print('\n >>>>> model 2: LSTM')
        model_lstm = create_model_lstm(X_train, optimizer='Nadam')
        history = fit_model(model_lstm,X_train, y_train, 30, 80, validation, patience)
        yhat_lstm = prediction(model_lstm,X_test)
        y_test_inverse, yhat_lstm_inverse = inverse_transform(y_test, yhat_lstm)

        # Bi-directional model .............................................................................................
        print('\n >>>>> model 3: Bi-directional')
        model_bilstm = create_model_bilstm(X_train, optimizer='adam')
        history = fit_model(model_bilstm,X_train, y_train, 100, 500, validation, patience)
        yhat_bilstm = prediction(model_bilstm,X_test)
        y_test_inverse, yhat_bilstm_inverse = inverse_transform(y_test, yhat_bilstm)

        # Encoder-decoder LSTM model .......................................................................................
        print('\n >>>>> model 4: Encoder-decoder LSTM')
        model_enc_dec = create_model_enc_dec(X_train, optimizer='Adamax')
        history = fit_model(model_enc_dec,X_train, y_train, 200, 10, validation, patience)
        yhat_endelstm = prediction(model_enc_dec,X_test)
        y_test_inverse, yhat_endelstm_inverse = inverse_transform(y_test, yhat_endelstm)

        # CNN-LSTM Encoder-Decoder model ...................................................................................
        print('\n >>>>> model 5: CNN-LSTM Encoder-Decoder')
        model_enc_dec_cnn = create_model_enc_dec_cnn(X_train, optimizer='Nadam',kern_size=3)
        history = fit_model(model_enc_dec_cnn, X_train, y_train, 20, 500, validation, patience)
        yhat_cnnlstmende = prediction(model_enc_dec_cnn,X_test)
        y_test_inverse, yhat_cnnlstmende_inverse = inverse_transform(y_test, yhat_cnnlstmende)

        # Vector-Output model .............................................................................................
        print('\n >>>>> model 6: Vector-Output')
        model_vector_output = create_model_vector_output(X_train, optimizer='Adam',kern_size=3)
        history = fit_model(model_vector_output, X_train, y_train, 50, 10, validation, patience)
        yhat_veout = prediction(model_vector_output,X_test)
        y_test_inverse, yhat_veout_inverse = inverse_transform(y_test, yhat_veout)

        # Multi-Head CNN-LSTM model ......................................................................................
        print('\n >>>>> model 7: Multi-Head CNN-LSTM')
        multi_head_cnn_lstm_model = create_model_multi_head_cnn_lstm(X_train, optimizer='Nadam',kern_size=3)
        history = fit_model(multi_head_cnn_lstm_model, X_train, y_train, 500, 40, validation, patience)
        yhat_muhecnnlstm = prediction(multi_head_cnn_lstm_model,X_test)
        y_test_inverse, yhat_muhecnnlstm_inverse = inverse_transform(y_test, yhat_muhecnnlstm)

            # GRU model .........................................................................................................
        print('\n >>>>> model 1: GRU')
        evaluate_forecast(y_test_inverse, yhat_gru_inverse)
        print('\n >>>>> model 2: LSTM')
        evaluate_forecast(y_test_inverse, yhat_lstm_inverse)
        print('\n >>>>> model 3: Bi-directional')
        evaluate_forecast(y_test_inverse, yhat_bilstm_inverse)
        print('\n >>>>> model 4: Encoder-decoder LSTM')
        evaluate_forecast(y_test_inverse, yhat_endelstm_inverse)
        print('\n >>>>> model 5: CNN-LSTM Encoder-Decoder')
        evaluate_forecast(y_test_inverse, yhat_cnnlstmende_inverse)
        print('\n >>>>> model 6: Vector-Output')
        evaluate_forecast(y_test_inverse, yhat_veout_inverse)
        print('\n >>>>> model 7: Multi-Head CNN-LSTM')
        evaluate_forecast(y_test_inverse, yhat_muhecnnlstm_inverse)

        # PLOT ----------------------------------------------------------------------------------------------------------
        # plot models
        plt.figure(figsize=(20, 10))
        plt.plot(pd.DataFrame(y_test_inverse)[[col_idx]], label='True Values', linewidth=2.5)
        plt.plot(pd.DataFrame(yhat_gru_inverse)[[col_idx]], label='Pred. GRU', linestyle='--')
        plt.plot(pd.DataFrame(yhat_lstm_inverse)[[col_idx]], label='Pred. LSTM', linestyle='--')
        plt.plot(pd.DataFrame(yhat_bilstm_inverse)[[col_idx]], label='Pred. BiLSTM', linestyle='--')
        plt.plot(pd.DataFrame(yhat_endelstm_inverse)[[col_idx]], label='Pred. Enc-Dec LSTM', linestyle='--')
        plt.plot(pd.DataFrame(yhat_cnnlstmende_inverse)[[col_idx]], label='Pred. CNN-LSTM Enc-Dec', linestyle='--')
        plt.plot(pd.DataFrame(yhat_veout_inverse)[[col_idx]], label='Pred. Vector Output', linestyle='--')
        plt.plot(pd.DataFrame(yhat_muhecnnlstm_inverse)[[col_idx]], label='Pred. Mult-Head CNN-LSTM', linestyle='--')
        plt.xlabel('Time')
        plt.ylabel('Value')
        plt.title('Real vs. Predicted Values MODELS')
        plt.legend()
        plt.show()

## Temporality data

In [ ]:
# Objective and predictors ...................................
objective = "timeseries_350"
cc_predictors = ['timeseries_405','timeseries_363','timeseries_975','timeseries_120','timeseries_2','timeseries_541','timeseries_775',
                 'timeseries_181','timeseries_692','timeseries_443','timeseries_763','timeseries_31','timeseries_324'] 

# df definition ..............................................
cc_predictors.append(objective)
variables_predictors = cc_predictors.copy()
subdf = smoothed[cc_predictors]

# Add temporality (weekday and month) ........................
temporality = False
if temporality:
    subdf['weekday'] = subdf['date'].dt.day_name()
    subdf['month'] = subdf['date'].dt.month_name()
    dummies_day = pd.get_dummies(subdf['weekday'], prefix='weekday_')
    dummies_month = pd.get_dummies(subdf['month'], prefix='month_')
    dummies_day = dummies_day.astype(int)
    dummies_month = dummies_month.astype(int)
    subdf = pd.concat([subdf, dummies_day, dummies_month], axis=1)
    subdf = subdf.drop('weekday', axis = 1)
    subdf = subdf.drop('month', axis = 1)
    cc_predictors = cc_predictors + dummies_day.columns.tolist()
    cc_predictors = cc_predictors + dummies_month.columns.tolist()
    cc_predictors.remove('date')


# put objective to last column ..............................
cc_predictors = [col for col in subdf.columns if col != objective]
cc_predictors.append(objective)
subdf = subdf[cc_predictors]
display(subdf) 

# train-test split ...........................................
train_size = int(len(subdf)*0.8)
train_dataset, test_dataset = subdf.iloc[5:train_size],subdf.iloc[train_size:]

#scale ......................................................
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train_dataset)
scaled_test = scaler.transform(test_dataset)

# get idx of objective variable
col_idx = int(subdf.columns.get_loc(objective))-1

FORECAST = [1,7,14,21,28]                 # quants dies em de predir
n_features = len(cc_predictors)   # quantes variables (sistema molt sensible)

validation = 0.1
patience = 10
LOOK_BACKS = [7,14,30,60,90,180,365,730]

#results_df
columns = ['Algorithm', 'look_back', 'forecast_range', 'mae', 'mse', 'mape']  
result = pd.DataFrame(columns=columns)

for LOOK_BACK in LOOK_BACKS:
    for FORECAST_RANGE in FORECAST: 
    
        LOOK_BACK = int(LOOK_BACK)
        FORECAST_RANGE = int(FORECAST_RANGE)
        print("\n \n \n >>>> LOOK BACK PARAMETER: %s" % (LOOK_BACK))
        print(">>>> FORECAST RANGE PARAMETER: %s" % (FORECAST_RANGE))

        X_train, y_train = split_sequence(scaled_train, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
        X_test, y_test = split_sequence(scaled_test, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)

        # GRU model .........................................................................................................
        print('\n >>>>> model 1: GRU')
        model_gru = create_model_gru(X_train, optimizer='Nadam')
        history = fit_model(model_gru,X_train, y_train,100, 80, validation, patience)
        yhat_gru = prediction(model_gru,X_test)
        y_test_inverse, yhat_gru_inverse = inverse_transform(y_test, yhat_gru)

        # LSTM model ........................................................................................................
        print('\n >>>>> model 2: LSTM')
        model_lstm = create_model_lstm(X_train, optimizer='Nadam')
        history = fit_model(model_lstm,X_train, y_train, 30, 80, validation, patience)
        yhat_lstm = prediction(model_lstm,X_test)
        y_test_inverse, yhat_lstm_inverse = inverse_transform(y_test, yhat_lstm)

        # Bi-directional model .............................................................................................
        print('\n >>>>> model 3: Bi-directional')
        model_bilstm = create_model_bilstm(X_train, optimizer='adam')
        history = fit_model(model_bilstm,X_train, y_train, 100, 500, validation, patience)
        yhat_bilstm = prediction(model_bilstm,X_test)
        y_test_inverse, yhat_bilstm_inverse = inverse_transform(y_test, yhat_bilstm)

        # Encoder-decoder LSTM model .......................................................................................
        print('\n >>>>> model 4: Encoder-decoder LSTM')
        model_enc_dec = create_model_enc_dec(X_train, optimizer='Adamax')
        history = fit_model(model_enc_dec,X_train, y_train, 200, 10, validation, patience)
        yhat_endelstm = prediction(model_enc_dec,X_test)
        y_test_inverse, yhat_endelstm_inverse = inverse_transform(y_test, yhat_endelstm)

        # CNN-LSTM Encoder-Decoder model ...................................................................................
        print('\n >>>>> model 5: CNN-LSTM Encoder-Decoder')
        model_enc_dec_cnn = create_model_enc_dec_cnn(X_train, optimizer='Nadam',kern_size=3)
        history = fit_model(model_enc_dec_cnn, X_train, y_train, 20, 500, validation, patience)
        yhat_cnnlstmende = prediction(model_enc_dec_cnn,X_test)
        y_test_inverse, yhat_cnnlstmende_inverse = inverse_transform(y_test, yhat_cnnlstmende)

        # Vector-Output model .............................................................................................
        print('\n >>>>> model 6: Vector-Output')
        model_vector_output = create_model_vector_output(X_train, optimizer='Adam',kern_size=3)
        history = fit_model(model_vector_output, X_train, y_train, 50, 10, validation, patience)
        yhat_veout = prediction(model_vector_output,X_test)
        y_test_inverse, yhat_veout_inverse = inverse_transform(y_test, yhat_veout)

        # Multi-Head CNN-LSTM model ......................................................................................
        print('\n >>>>> model 7: Multi-Head CNN-LSTM')
        multi_head_cnn_lstm_model = create_model_multi_head_cnn_lstm(X_train, optimizer='Nadam',kern_size=3)
        history = fit_model(multi_head_cnn_lstm_model, X_train, y_train, 500, 40, validation, patience)
        yhat_muhecnnlstm = prediction(multi_head_cnn_lstm_model,X_test)
        y_test_inverse, yhat_muhecnnlstm_inverse = inverse_transform(y_test, yhat_muhecnnlstm)

            # GRU model .........................................................................................................
        print('\n >>>>> model 1: GRU')
        evaluate_forecast(y_test_inverse, yhat_gru_inverse)
        res = get_results(y_test_inverse, yhat_gru_inverse)
        new_row = pd.DataFrame([{'Algorithm': 'GRU', 'look_back': LOOK_BACK, 'forecast_range': FORECAST_RANGE, 
                   'mae': res[0], 'mse': res[1], 'mape':res[2]}])
        result = pd.concat([result, new_row], ignore_index=True)
        print('\n >>>>> model 2: LSTM')
        evaluate_forecast(y_test_inverse, yhat_lstm_inverse)
        res = get_results(y_test_inverse, yhat_lstm_inverse)
        new_row = pd.DataFrame([{'Algorithm': 'LSTM', 'look_back': LOOK_BACK, 'forecast_range': FORECAST_RANGE, 
                   'mae': res[0], 'mse': res[1], 'mape':res[2]}])
        result = pd.concat([result, new_row], ignore_index=True)
        print('\n >>>>> model 3: Bi-directional')
        evaluate_forecast(y_test_inverse, yhat_bilstm_inverse)
        res = get_results(y_test_inverse, yhat_bilstm_inverse)
        new_row = pd.DataFrame([{'Algorithm': 'Bi-directional', 'look_back': LOOK_BACK, 'forecast_range': FORECAST_RANGE, 
                   'mae': res[0], 'mse': res[1], 'mape':res[2]}])
        result = pd.concat([result, new_row], ignore_index=True)
        print('\n >>>>> model 4: Encoder-decoder LSTM')
        evaluate_forecast(y_test_inverse, yhat_endelstm_inverse)
        res = get_results(y_test_inverse, yhat_endelstm_inverse)
        new_row = pd.DataFrame([{'Algorithm': 'Encoder-decoder LSTM', 'look_back': LOOK_BACK, 'forecast_range': FORECAST_RANGE, 
                   'mae': res[0], 'mse': res[1], 'mape':res[2]}])
        result = pd.concat([result, new_row], ignore_index=True)
        print('\n >>>>> model 5: CNN-LSTM Encoder-Decoder')
        evaluate_forecast(y_test_inverse, yhat_cnnlstmende_inverse)
        res = get_results(y_test_inverse, yhat_cnnlstmende_inverse)
        new_row = pd.DataFrame([{'Algorithm': 'CNN-LSTM Encoder-Decoder', 'look_back': LOOK_BACK, 'forecast_range': FORECAST_RANGE, 
                   'mae': res[0], 'mse': res[1], 'mape':res[2]}])
        result = pd.concat([result, new_row], ignore_index=True)
        print('\n >>>>> model 6: Vector-Output')
        evaluate_forecast(y_test_inverse, yhat_veout_inverse)
        res = get_results(y_test_inverse, yhat_veout_inverse)
        new_row = pd.DataFrame([{'Algorithm': 'Vector-Output', 'look_back': LOOK_BACK, 'forecast_range': FORECAST_RANGE, 
                   'mae': res[0], 'mse': res[1], 'mape':res[2]}])
        result = pd.concat([result, new_row], ignore_index=True)
        print('\n >>>>> model 7: Multi-Head CNN-LSTM')
        evaluate_forecast(y_test_inverse, yhat_muhecnnlstm_inverse)
        res = get_results(y_test_inverse, yhat_muhecnnlstm_inverse)
        new_row = pd.DataFrame([{'Algorithm': 'Multi-Head CNN-LSTM', 'look_back': LOOK_BACK, 'forecast_range': FORECAST_RANGE, 
                   'mae': res[0], 'mse': res[1], 'mape':res[2]}])
        result = pd.concat([result, new_row], ignore_index=True)

        # PLOT ----------------------------------------------------------------------------------------------------------
        # plot models
        plt.figure(figsize=(20, 10))
        plt.plot(pd.DataFrame(y_test_inverse)[[col_idx]], label='True Values', linewidth=2.5)
        plt.plot(pd.DataFrame(yhat_gru_inverse)[[col_idx]], label='Pred. GRU', linestyle='--')
        plt.plot(pd.DataFrame(yhat_lstm_inverse)[[col_idx]], label='Pred. LSTM', linestyle='--')
        plt.plot(pd.DataFrame(yhat_bilstm_inverse)[[col_idx]], label='Pred. BiLSTM', linestyle='--')
        plt.plot(pd.DataFrame(yhat_endelstm_inverse)[[col_idx]], label='Pred. Enc-Dec LSTM', linestyle='--')
        plt.plot(pd.DataFrame(yhat_cnnlstmende_inverse)[[col_idx]], label='Pred. CNN-LSTM Enc-Dec', linestyle='--')
        plt.plot(pd.DataFrame(yhat_veout_inverse)[[col_idx]], label='Pred. Vector Output', linestyle='--')
        plt.plot(pd.DataFrame(yhat_muhecnnlstm_inverse)[[col_idx]], label='Pred. Mult-Head CNN-LSTM', linestyle='--')
        plt.xlabel('Time')
        plt.ylabel('Value')
        plt.title('Real vs. Predicted Values MODELS (FORECAST RANGE: %s // LOOK BACK: %s)'%(FORECAST_RANGE,LOOK_BACK))
        plt.legend()
        plt.show()
        
        display(result)

In [ ]:
# Convert the "mae," "mse," and "mape" columns to numeric if they are not already
result_df['mae'] = pd.to_numeric(result_df['mae'])
result_df['mse'] = pd.to_numeric(result_df['mse'])
result_df['mape'] = pd.to_numeric(result_df['mape'])
result_df

#result_df.to_excel("result_forecast_look.xlsx", index=False)

In [ ]:
# Create line plots
plt.figure(figsize=(10, 6))

# Using Seaborn for color-coding by algorithm
sns.lineplot(data=result_df, x='look_back', y='forecast_range', hue='Algorithm', style='Algorithm', markers=True)

# Set labels and title
plt.xlabel('Look Back')
plt.ylabel('Forecast Range')
plt.title('Line Plots of Look Back vs. Forecast Range by Algorithm')

# Show the legend
plt.legend(title='Algorithm')

# Display the plot
plt.show()

# ARIMA MODEL

In [ ]:
FORECAST_RANGE = 7                 # quants dies em de predir
n_features = len(cc_predictors)   # quantes variables (sistema molt sensible)

validation = 0.1
patience = 10
LOOK_BACKS = [7,14,30,60,90,180,365,730]
LOOK_BACK = 7
#for LOOK_BACK in LOOK_BACKS:
    
    
# train-test split ...........................................
train_size = int(len(subdf)*0.8)
train_dataset, test_dataset = subdf.iloc[5:train_size],subdf.iloc[train_size:]
tr_ds = train_dataset[[objective]]
ts_ds = test_dataset[[objective]]



    
print("\n \n \n >>>> LOOK BACK PARAMETER: %s" % (LOOK_BACK))
    
X_train, y_train = split_sequence(tr_ds, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
X_test, y_test = split_sequence(ts_ds, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)

   
print('\n >>>>> model 7: ARIMA')
for i in range(len(X_train)):
    arima_model = ARIMA(X_train[i], order=(2,1,3))
    model = arima_model.fit()
    yhat_arima = model.predict(start=0, end=FORECAST_RANGE)
    evaluate_forecast(y_test, yhat_arima)

    # PLOT ----------------------------------------------------------------------------------------------------------
    # plot models
plt.figure(figsize=(20, 10))
plt.plot(pd.DataFrame(y_test), label='True Values', linewidth=2.5)
plt.plot(pd.DataFrame(yhat_arima), label='Pred. GRU', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Real vs. Predicted Values MODELS')
plt.legend()
plt.show()

mae: tf.Tensor(86.08014291016593, shape=(), dtype=float64)
mse: tf.Tensor(8381.30922365042, shape=(), dtype=float64)
mape: tf.Tensor(76.92132133493939, shape=(), dtype=float64)
mae: tf.Tensor(83.62846368233456, shape=(), dtype=float64)
mse: tf.Tensor(7992.263635805765, shape=(), dtype=float64)
mape: tf.Tensor(74.8479748370286, shape=(), dtype=float64)
mae: tf.Tensor(82.00350422941699, shape=(), dtype=float64)
mse: tf.Tensor(7775.590951652088, shape=(), dtype=float64)
mape: tf.Tensor(73.4737722281768, shape=(), dtype=float64)
mae: tf.Tensor(82.63602174619058, shape=(), dtype=float64)
mse: tf.Tensor(7852.906556171393, shape=(), dtype=float64)
mape: tf.Tensor(74.00868232709284, shape=(), dtype=float64)
mae: tf.Tensor(83.28299589439526, shape=(), dtype=float64)
mse: tf.Tensor(7951.13140362937, shape=(), dtype=float64)
mape: tf.Tensor(74.55581817145267, shape=(), dtype=float64)
mae: tf.Tensor(83.1233366074593, shape=(), dtype=float64)
mse: tf.Tensor(7919.507360100786, shape=(), dtype=float6

mae: tf.Tensor(48.71401039664185, shape=(), dtype=float64)
mse: tf.Tensor(3830.1243474150347, shape=(), dtype=float64)
mape: tf.Tensor(44.92446134346418, shape=(), dtype=float64)
mae: tf.Tensor(47.91468092297391, shape=(), dtype=float64)
mse: tf.Tensor(3756.712181524979, shape=(), dtype=float64)
mape: tf.Tensor(44.21428993003127, shape=(), dtype=float64)
mae: tf.Tensor(46.235657446118395, shape=(), dtype=float64)
mse: tf.Tensor(3612.1817237361356, shape=(), dtype=float64)
mape: tf.Tensor(42.661015705332375, shape=(), dtype=float64)
mae: tf.Tensor(46.328157237195114, shape=(), dtype=float64)
mse: tf.Tensor(3618.704183645121, shape=(), dtype=float64)
mape: tf.Tensor(42.76319835625442, shape=(), dtype=float64)
mae: tf.Tensor(47.41248062672275, shape=(), dtype=float64)
mse: tf.Tensor(3712.932137892238, shape=(), dtype=float64)
mape: tf.Tensor(43.758895632590495, shape=(), dtype=float64)
mae: tf.Tensor(48.260649023207954, shape=(), dtype=float64)
mse: tf.Tensor(3786.8526210726045, shape=(),

mae: tf.Tensor(38.26010356302696, shape=(), dtype=float64)
mse: tf.Tensor(2890.760879623729, shape=(), dtype=float64)
mape: tf.Tensor(30.622582956222544, shape=(), dtype=float64)
mae: tf.Tensor(38.24846175115859, shape=(), dtype=float64)
mse: tf.Tensor(2887.04909103886, shape=(), dtype=float64)
mape: tf.Tensor(30.57853589560393, shape=(), dtype=float64)
mae: tf.Tensor(38.028900947658045, shape=(), dtype=float64)
mse: tf.Tensor(2877.1186486458396, shape=(), dtype=float64)
mape: tf.Tensor(30.532529935323247, shape=(), dtype=float64)
mae: tf.Tensor(37.95660525320669, shape=(), dtype=float64)
mse: tf.Tensor(2870.80499961044, shape=(), dtype=float64)
mape: tf.Tensor(30.47905269933552, shape=(), dtype=float64)
mae: tf.Tensor(37.727647482881885, shape=(), dtype=float64)
mse: tf.Tensor(2859.470452192583, shape=(), dtype=float64)
mape: tf.Tensor(30.415719045104865, shape=(), dtype=float64)
mae: tf.Tensor(37.12548414028118, shape=(), dtype=float64)
mse: tf.Tensor(2811.508334623012, shape=(), dty

mae: tf.Tensor(57.103243484659394, shape=(), dtype=float64)
mse: tf.Tensor(4643.5589303572715, shape=(), dtype=float64)
mape: tf.Tensor(52.286415975455455, shape=(), dtype=float64)
mae: tf.Tensor(59.235265709688676, shape=(), dtype=float64)
mse: tf.Tensor(4840.208344847781, shape=(), dtype=float64)
mape: tf.Tensor(54.16122528636982, shape=(), dtype=float64)
mae: tf.Tensor(61.76941567393895, shape=(), dtype=float64)
mse: tf.Tensor(5109.647421813738, shape=(), dtype=float64)
mape: tf.Tensor(56.33136887866268, shape=(), dtype=float64)
mae: tf.Tensor(63.718977320647014, shape=(), dtype=float64)
mse: tf.Tensor(5359.0317795112305, shape=(), dtype=float64)
mape: tf.Tensor(57.96487058169311, shape=(), dtype=float64)
mae: tf.Tensor(65.65231471232893, shape=(), dtype=float64)
mse: tf.Tensor(5603.40737903642, shape=(), dtype=float64)
mape: tf.Tensor(59.60074069434732, shape=(), dtype=float64)
mae: tf.Tensor(68.16818567829432, shape=(), dtype=float64)
mse: tf.Tensor(5911.856234932615, shape=(), dt

mae: tf.Tensor(98.76913757656334, shape=(), dtype=float64)
mse: tf.Tensor(10611.346524427352, shape=(), dtype=float64)
mape: tf.Tensor(87.65220421510125, shape=(), dtype=float64)
mae: tf.Tensor(96.84199294498127, shape=(), dtype=float64)
mse: tf.Tensor(10233.878285285373, shape=(), dtype=float64)
mape: tf.Tensor(86.02244836143977, shape=(), dtype=float64)
mae: tf.Tensor(90.68720429203489, shape=(), dtype=float64)
mse: tf.Tensor(9216.07833503554, shape=(), dtype=float64)
mape: tf.Tensor(80.81744056347625, shape=(), dtype=float64)
mae: tf.Tensor(90.85360171674242, shape=(), dtype=float64)
mse: tf.Tensor(9209.166993501947, shape=(), dtype=float64)
mape: tf.Tensor(80.95816024403737, shape=(), dtype=float64)
mae: tf.Tensor(89.60837850573739, shape=(), dtype=float64)
mse: tf.Tensor(8984.393675320844, shape=(), dtype=float64)
mape: tf.Tensor(79.90509459240302, shape=(), dtype=float64)
mae: tf.Tensor(90.18499889092305, shape=(), dtype=float64)
mse: tf.Tensor(9066.113696277345, shape=(), dtype=

...

mae: tf.Tensor(42.914207169490105, shape=(), dtype=float64)
mse: tf.Tensor(3223.392590872775, shape=(), dtype=float64)
mape: tf.Tensor(33.27880794563174, shape=(), dtype=float64)
mae: tf.Tensor(43.5226708917287, shape=(), dtype=float64)
mse: tf.Tensor(3269.816822335066, shape=(), dtype=float64)
mape: tf.Tensor(33.65676066417775, shape=(), dtype=float64)
mae: tf.Tensor(43.835860753540715, shape=(), dtype=float64)
mse: tf.Tensor(3292.9931914994386, shape=(), dtype=float64)
mape: tf.Tensor(33.84661800844532, shape=(), dtype=float64)
mae: tf.Tensor(44.56480860780855, shape=(), dtype=float64)
mse: tf.Tensor(3352.439705597448, shape=(), dtype=float64)
mape: tf.Tensor(34.32271203424271, shape=(), dtype=float64)
mae: tf.Tensor(45.45760371328932, shape=(), dtype=float64)
mse: tf.Tensor(3435.574731880813, shape=(), dtype=float64)
mape: tf.Tensor(34.938692181611636, shape=(), dtype=float64)
mae: tf.Tensor(46.05950846734229, shape=(), dtype=float64)
mse: tf.Tensor(3486.212189799515, shape=(), dtyp

mae: tf.Tensor(46.53755389828254, shape=(), dtype=float64)
mse: tf.Tensor(3528.5041696789713, shape=(), dtype=float64)
mape: tf.Tensor(35.65542245583057, shape=(), dtype=float64)
mae: tf.Tensor(46.16825601458204, shape=(), dtype=float64)
mse: tf.Tensor(3488.221922971375, shape=(), dtype=float64)
mape: tf.Tensor(35.38382957984447, shape=(), dtype=float64)
mae: tf.Tensor(45.126861241850825, shape=(), dtype=float64)
mse: tf.Tensor(3399.5748091385, shape=(), dtype=float64)
mape: tf.Tensor(34.692755472459844, shape=(), dtype=float64)
mae: tf.Tensor(43.881142777604765, shape=(), dtype=float64)
mse: tf.Tensor(3304.5470181437986, shape=(), dtype=float64)
mape: tf.Tensor(33.91634937295653, shape=(), dtype=float64)
mae: tf.Tensor(43.38805498651625, shape=(), dtype=float64)
mse: tf.Tensor(3264.1462300198937, shape=(), dtype=float64)
mape: tf.Tensor(33.594013831244204, shape=(), dtype=float64)
mae: tf.Tensor(42.62751372696996, shape=(), dtype=float64)
mse: tf.Tensor(3208.709971118364, shape=(), dt

mae: tf.Tensor(34.818749436790974, shape=(), dtype=float64)
mse: tf.Tensor(2692.5770185347033, shape=(), dtype=float64)
mape: tf.Tensor(30.127206042315382, shape=(), dtype=float64)
mae: tf.Tensor(35.053504014882634, shape=(), dtype=float64)
mse: tf.Tensor(2708.7593866212924, shape=(), dtype=float64)
mape: tf.Tensor(30.44704837784266, shape=(), dtype=float64)
mae: tf.Tensor(34.97630085029628, shape=(), dtype=float64)
mse: tf.Tensor(2702.896050970544, shape=(), dtype=float64)
mape: tf.Tensor(30.28597154389773, shape=(), dtype=float64)
mae: tf.Tensor(34.83766051233821, shape=(), dtype=float64)
mse: tf.Tensor(2697.2854974787265, shape=(), dtype=float64)
mape: tf.Tensor(30.390166131944824, shape=(), dtype=float64)
mae: tf.Tensor(34.87982749502933, shape=(), dtype=float64)
mse: tf.Tensor(2703.108052819107, shape=(), dtype=float64)
mape: tf.Tensor(30.653184832499043, shape=(), dtype=float64)
mae: tf.Tensor(35.42090074140069, shape=(), dtype=float64)
mse: tf.Tensor(2741.83177573709, shape=(), 

ValueError: Must pass 2-d input. shape=(718, 7, 1)

<Figure size 2000x1000 with 0 Axes>

In [ ]:
# default libs
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Forcesting libs
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pandas.plotting import register_matplotlib_converters

# default lib for ignore warinigs
import warnings
warnings.filterwarnings('ignore')

This summarizes the coefficient values used as well as the skill of the fit on the on the in-sample observations.
Non-seasonal ARIMA models are generally denoted ARIMA(p,d,q) where parameters p, d, and q are non-negative integers, p is the order (number of time lags) of the auto-regressive model, d is the degree of differencing (the number of times the data have had past values subtracted), and q is the order of the moving-average model. Seasonal ARIMA models are usually denoted ARIMA(p,d,q)(P,D,Q)m, where m refers to the number of periods in each season, and the uppercase P,D,Q refer to the auto-regressive, differencing, and moving average terms for the seasonal part of the ARIMA model.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
series = smoothed[['timeseries_350']]

arima_model = ARIMA(series.values, order=(2,1,3))
model = arima_model.fit()
print(model.summary())

# Get the predicted values
predictions = model.predict(start=0, end=len(series) - 1, typ='levels')

In [ ]:
# Plot actual vs. predicted values
plt.figure(figsize=(12, 6))
plt.plot(series['I10'].values, label='Actual')
plt.plot(predictions, label='Predicted', linestyle='--')
plt.legend()
plt.title('Actual vs. Predicted Values')
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()